In [1]:
# Copyright 2021 by Marc Fiammante, marc.fiammante@free.fr
# All rights reserved.
# This file a neural network label file generator, specifically the Pascal VOC format 
# http://host.robots.ox.ac.uk/pascal/VOC/
# and is released under the "MIT License Agreement". Please see the LICENSE
# file that should have been included as part of this package.

import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
import cv2
import numpy as np
import io
import os
import copy

# draw a rectangle on an image
def draw(img,rects,color):
    for r in rects:
        p1 = (r[0], r[1])
        p2 = (r[0]+r[2], r[1]+r[3])
        cv2.rectangle(img, p1,p2, color,10)
        
# remove duplicates in a list of tuples
def removeDuplicates(lst):     
    return [t for t in (set(tuple(i) for i in lst))]

# union of two rectangles
def union(a,b):
  x = min(a[0], b[0])
  y = min(a[1], b[1])
  w = max(a[0]+a[2], b[0]+b[2]) - x
  h = max(a[1]+a[3], b[1]+b[3]) - y
  return (x, y, w, h)

# intersection of two rectangles
def intersection(a,b,margin):
  x = max(a[0]-margin, b[0]-margin)
  y = max(a[1]-margin, b[1]-margin)
  w = min(a[0]+a[2]+margin, b[0]+b[2]+margin) - x 
  h = min(a[1]+a[3]+margin, b[1]+b[3]+margin) - y 
  if w<0 or h<0: return None # 
  return (x, y, w, h)
#parse args
def parse_args():
    parser=argparse.ArgumentParser(description="a script to do create PascalVOC files from mask and objectid")
    parser.add_argument('-mask', required=True,help="path of mask file")
    parser.add_argument('-ids', required=True,help="path of id file")
    parser.add_argument('-image', required=False,help="optional path of image file")
    parser.add_argument('-pascalvoc', required=False,help="optional path of pascal VOC file, default is name of id file with voc extension")
    args=parser.parse_args()
    return args
# copy XML tree of an element
def copy_tree( tree_root ):
    return et.ElementTree( tree_root );
#==============================================================================
# process mask an dobjectid file to create bounding boxes for Pascal VOC format
def main():
    if not inIPython:
        inputs=parse_args
        idfile=args.ids        
        imagefile=args.image    
        maskfile=args.mask    
    else:
        idfile ='C:/projets/oktal-se/objectid.txt'
        imagefile='C:/projets/oktal-se/D_VIS_1_30x10_B_M_000.png'
        maskfile='C:/projets/oktal-se/D_VIS_1_30x10_B_M_000_mask.png'
    mask_head_tail = os.path.split(maskfile)
    maskpath=mask_head_tail[0]
    maskname=mask_head_tail[1]

    objectid = open(idfile, 'r')
    objects = objectid.readlines()
    rgbis=[]
    for object in objects:
        (r,g,b,id) = object.split()
        rgbis.append((int(b),int(g),int(r),int(id)))

    mask = cv2.imread(maskfile)
    imgsyn = cv2.imread(imagefile)
    rows,cols,depth=mask.shape
    pixels=rows*cols
    boundingboxes=dict()
    rgbiskeep=[]
    for i,rgbi in enumerate(rgbis):
        color=np.array(rgbi[0:3])
        indices = np.where(np.any(mask != color, axis=-1))
        count=pixels-len(indices[0])
        percent=int(100*count/pixels)
        if percent<15:
            print("color",color)
            mask_copy = mask.copy()
            mask_copy[indices] = [0, 0, 0]
            maskgray = cv2.cvtColor(mask_copy, cv2.COLOR_BGR2GRAY)
            contours, hierarchy = cv2.findContours(maskgray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # merge overlapping contours
            if len(contours)>0:
                rects = []
                for cnt in contours:
                    x,y,w,h = cv2.boundingRect(cnt)
                    rects.append([x, y, w, h])
                process=True
                while process:
                    process=False
                    for irect1 in range(len(rects)-1):
                        for irect2 in range(irect1+1,len(rects)):
                            if intersection(rects[irect1],rects[irect2],5)!=None:
                                rects[irect1]=union(rects[irect1],rects[irect2])
                                rects[irect2]=union(rects[irect1],rects[irect2])
                                process=True
                    rects=removeDuplicates(rects)
                draw(imgsyn,rects, (int(color[0]),int(color[1]),int(color[2])))
                for rect in rects:
                    print(rect)
                boundingboxes[str(rgbi[3])]=rects
        
        cv2.imwrite(os.path.join(maskpath, "mask+box.png"), imgsyn)

    # now create Pascal VOC
    xmlvocstring="<annotation><folder>folder</folder><filename>filename</filename>"
    xmlvocstring+="<source><database>OKTAL-SE</database><annotation>PASCAL VOC2008</annotation><image>simulation</image></source>"
    xmlvocstring+="<size><width>1000</width><height>1000</height><depth>3</depth><segmented>0</segmented></size></annotation>"
    xmlobjstring="<object><name>type1</name><pose>Unspecified</pose><truncated>0</truncated><occluded>0</occluded><difficult>0</difficult></object>"
    xmlbndstring="<bndbox><xmin>0</xmin><ymin>0</ymin><xmax>100</xmax><ymax>100</ymax></bndbox>"                            
                              
    fvoc= io.StringIO(xmlvocstring)
    tree = ET.parse(fvoc)
    root = tree.getroot()
    count=0
    for child in root:
        print("child",child.tag)
        if child.tag=="filename":
            child.text=maskname
        elif child.tag=="size":
            for schild in child:
                if schild.tag=="width":
                    schild.text=str(cols)
                elif schild.tag=="height":
                    schild.text=str(rows)
        count+=1
    for objectid in boundingboxes:
        fobj= io.StringIO(xmlobjstring)
        objtree = ET.parse(fobj)
        vobject = objtree.getroot()
        for name in vobject.findall('name'):
            name.text=str(objectid)
        rects=boundingboxes[objectid]
        for rect in rects:
            fbnd= io.StringIO(xmlbndstring)
            bndtree = ET.parse(fbnd)
            vfbnd = bndtree.getroot()
            for bchild in vfbnd:
                if bchild.tag=="xmin":
                    bchild.text=str(rect[0])
                elif bchild.tag=="ymin":
                    bchild.text=str(rect[1])
                if bchild.tag=="xmax":
                    bchild.text=str(rect[0]+rect[2])
                elif bchild.tag=="ymax":
                    bchild.text=str(rect[1]+rect[3])
            vobject.append(vfbnd)                 
        
        root.insert(count,vobject) 
        count+=1
            
            
    ET.indent(tree, space="\t", level=0)
    tree.write(os.path.join(maskpath, maskname+"_pascalvoc.txt"))

if __name__ == '__main__':
    # try to detect that I'm in a notebook
    try:
        __IPYTHON__
    except NameError:
        inIPython = False
    else:
        inIPython = True
    main()

ModuleNotFoundError: No module named 'cv2'

In [1]:
from platform import python_version

print(python_version())

3.8.12
